# **Pretraining an LLM on Unlabeled Data**

In [2]:
from importlib.metadata import version

pkgs = ["matplotlib", 
        "numpy", 
        "tiktoken", 
        "torch",
        "tensorflow" # For OpenAI's pretrained weights since earlier models were trained in TensorFlow
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

matplotlib version: 3.10.1
numpy version: 2.2.5
tiktoken version: 0.9.0
torch version: 2.7.0
tensorflow version: 2.19.0


## **1. Evaluating Generative Text Models**

--prose--

In [3]:
import torch
from utils.components import GPTModel

# GPT configuration from previos NBs
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 1024, # Retaining original context length
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval(); # Dropout disabled during inference

In [4]:
import tiktoken
from utils.components import generate_text_simple

# Pipeline functions
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # Add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # Removing batch dim
    return tokenizer.decode(flat.tolist())

start_content = "In the grim darkness of the far future"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_content, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 In the grim darkness of the far futureAnimation inconvenient portion312atsHealth McAuliffedisplay marched negativity


### **1.1 Calculating The Text Generation Loss i.e. Cross-Entropy and Perplexity**

In [5]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [6]:
with torch.no_grad():
    logits = model(inputs)
    
probs = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probs.shape) # Shape ==> (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


In [8]:
token_ids = torch.argmax(probs, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[36397],
         [39619],
         [20610]],

        [[ 8615],
         [49289],
         [47105]]])


In [9]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  Gathering SerbianFriday


In [10]:
# Token probabilities corresponding to the target indices
text_idx = 0
target_probs_1 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probs_1)

text_idx = 1
target_probs_2 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probs_2)

Text 1: tensor([2.3466e-05, 2.0531e-05, 1.1733e-05])
Text 1: tensor([4.2794e-05, 1.6248e-05, 1.1586e-05])


In [11]:
# Compute log of all token probabilities
log_probs = torch.log(torch.cat((target_probs_1, target_probs_2)))
print(log_probs)

tensor([-10.6600, -10.7936, -11.3531, -10.0591, -11.0276, -11.3658])


In [12]:
# Computing the average log probability of each token
avg_log_probs = torch.mean(log_probs)
print(avg_log_probs)

tensor(-10.8765)


In [13]:
# Minimize the negative average log-probability
neg_avg_log_prob = avg_log_probs * -1
print(neg_avg_log_prob)

tensor(10.8765)


In [14]:
# Prior to implementing cross entropy, we should check the shape of logits and targets
# Logits shape --> (batch_size, num_tokens, vocab_size)
print("Logits shape:", logits.shape)

# Targets have shape (batch_size, num_tokens)
print("Targets shape:", targets.shape)

Logits shape: torch.Size([2, 3, 50257])
Targets shape: torch.Size([2, 3])


In [15]:
# Flattening above tensors for cross entropy. Flattening is done by combining over the batch dimension
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

# Applying cross entropy loss
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)

# Printing shapes and cross entropy loss
print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)
print("Cross-Entropy loss:", loss)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])
Cross-Entropy loss: tensor(10.8765)


In [16]:
# Calculating the perplexity 
perplexity = torch.exp(loss)
print("Perplexity:", perplexity)

Perplexity: tensor(52918.7773)
